<img src="images/borehole_graph.png" ></img>

<br>

# Borehole Graph Data Model Example

In this notebook, we will use <a href="https://steno3d.com">Steno3D</a> to view and share data throughout the steps of a lithological borehole graph from neo4j. 

## Login to Steno3D

The first thing you need to do is login to Steno3D. You can <a href="https://steno3d.com/signup">sign up for an account</a> to get your own developer API key if you do not have one already. Running the cell below will provide you with instructions for how to obtain and enter your key.

In [1]:
import steno3d

In [5]:
steno3d.login()


You are already logged in as @mmorley. To log in as a different user
please `steno3d.logout()`, then login specifying a different
username or API developer key.




In [59]:
prelim_data = steno3d.Project(
    title='Neo4j Borehole Experiment',
    description='An experiment in modeling 3d borehole data with neo4j.',
    public=True
)

## Query the Drill Collar Locations from Neo4j using Neo4j Driver

First step is to use a cypher query to extract the data from Neo4j. We will use the Neo4j Python driver for this, through the jupyter notebook. 

The Steno3d app requires collar data to be in a nested array of form: [[x,y,z]...[x,y,z]], so we require a neo4j query that returns the data through the python driver. 

One thing to note is that Steno3d uses cartesian cordinates, so we will use the original NAD87 based locations instead of the Neo4j spatial coordinates/points, which is why we preserved these. Ahh the joys of spatial data!

match (c:Collar) with [c.E_10TM83,c.N_10TM83,c.Location.z] as location  return collect(location) as location

The results of this statement will be placed into a variable for steno. 

In [76]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

def borehole_data(tx):
    records=tx.run("match (c:Collar) "
                         "with [c.E_10TM83,c.N_10TM83,c.Location.z] as location "
                         "return collect(location) as location")
    return records
        
with driver.session() as session:
    results=session.read_transaction(borehole_data).data(0)
    nodes=[]
    i = 0
    for record in results:
        target= i
        i += 1
        for item in record['location']:
            nodes.append(item)
            source= i
            i+=i
    # print(nodes)
    drill_locations=steno3d.Mesh0D(vertices=nodes) 


We only need the locations of these points, no other data, so let's construct our <a href="https://python.steno3d.com/en/latest/content/api/resources/point.html">point</a> resource now. We will make the points orange so they stand out when plotted.

In [73]:
drill_points = steno3d.Point(
    project=prelim_data,
    title='Proposed Drill Locations',
    mesh=drill_locations,
    opts=dict(color='orange')
)

After uploading these points to our project on <a href="https://steno3d.com">steno3d.com</a>, let's print the url of our project to share.

In [74]:
drill_points.upload()

This project is PUBLIC. It is viewable by everyone.
Total progress:  55% - Uploading: project Neo4j Borehole Experiment
Complete!
https://steno3d.com/resource/point/qigc2YjiVGBua6REXo75


'https://steno3d.com/resource/point/qigc2YjiVGBua6REXo75'

In [75]:
print(prelim_data.url)

https://steno3d.com/app/oZXUDuSzFYrqWsTd0GpV


## Plot the Borehole Data

After discussing the preliminary data project online with collaborators, it was decided to go ahead with borehole drilling. Now the data is back and available to be explored with. Let's make a new project to contain the results of the drilling.

In [25]:
result_data = steno3d.Project(
    title='Neo4j Borehole Experiment',
    description='An experiment in modeling 3d borehole data with neo4j.',
    public=True
)

To display our borehole data, we first need a <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html#meshes">mesh 1D</a> based on their spatial geometry. This is imported from the Neo4j graph database model. The `view_type` option sets the lines mesh to initially display as fixed-width, extruded "tubes" rather than true 1D lines.

In [66]:
def borehole_data(tx):
    records=tx.run("match (c:Collar) "
                         "with [c.Location.x,c.Location.y,c.Location.z] as location "
                         "return collect(location) as location")
    return records
        
with driver.session() as session:
    results=session.read_transaction(borehole_data).data(0)
    nodes=[]
    i = 0
    for record in results:
        target= i
        i += 1
        for item in record['location']:
            nodes.append(item)
            source= i
            i+=i
    # print(nodes)
    borehole_vertices=steno3d.Mesh0D(vertices=nodes) 
    borehole_segments

borehole_mesh = steno3d.Mesh1D(
    vertices = borehole_vertices,
    segments = borehole_segments,
    opts = dict(
        view_type='boreholes'
    )
)

NameError: name 'Wolfpass' is not defined

Let's create a <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html">line</a> resource now using this mesh.

In [ ]:
boreholes = steno3d.Line(
    project=result_data,
    title='Borehole Results',
    description='Data based on results of borehole drilling',
    mesh=borehole_mesh
)

We need to add all the data to the boreholes now.

In [ ]:
borehole_raw_data = Wolfpass.borehole_data
borehole_data = [
    steno3d.DataArray(title=rd, array=borehole_raw_data[rd]) for rd in borehole_raw_data
]
borehole_bound_data = [
    dict(location='CC', data=d) for d in borehole_data
]

In [ ]:
boreholes.data = borehole_bound_data

These boreholes will be easier to analyze if the ground surface is also present. We can simply add the ground surface resource created for the <a href="#Inspect-the-Surface-Topography">preliminary data project</a> to our results data project.

Now our borehole results can be uploaded and shared.

In [24]:
result_data.upload()

NameError: name 'result_data' is not defined

## Explore the Modeled Formation

Based on the borehole results, modelling of the dacite formation has been carried out. Let's view this modeled surface with our other results using <a href="https://steno3d.com">Steno3D</a>. We will create, upload, and print the url for sharing in one step.

In [ ]:
dacite_ind = Wolfpass.lith_names.index('dacite')
dacite_vertices = Wolfpass.lith_vertices[dacite_ind]
dacite_triangles = Wolfpass.lith_triangles[dacite_ind]

dacite_surface = steno3d.Surface(
    project=result_data,
    title='Dacite Formation',
    description='Bounding surface of dacite based on borehole data and modelling results',
    mesh=steno3d.Mesh2D(
        vertices=dacite_vertices,
        triangles=dacite_triangles
    ),
    opts=dict(
        color='red'
    )
)
dacite_surface.upload()
print(result_data.url)

___
**Navigation**
- <a href="#Comprehensive-Example:-Wolf-Pass-Exploration-Project">Top of page</a>
- <a href="index.ipynb">Notebook home</a>
- <a href="https://steno3d.com">Steno3D website</a>
- <a href="https://steno3d.com/docs">Steno3D documentation</a>
- <a href="https://github.com/seequent/steno3d-notebooks/issues/new">Report an issue</a>
___